## Creating Rasterized Geodata for Least-Cost Path Analysis with PYORPS
This notebook demonstrates how to prepare rasterized geodata for use in least-cost path analysis with PYORPS. The
steps include downloading vector data from a WFS server, defining cost assumptions, and rasterizing the vector data
into a high-resolution raster dataset. The resulting raster file can then be used for route planning tasks with PYORPS.

#### 1. Import the features for cost assumptions and Rasterization from PYORPS
The initialize_geo_dataset function creates a GeoDataset Object from a given input.
The function 'save_empty_cost_assumptions' is used to save a blank file which can be filled with cost assumptions and
further be used for rasterization. Objects of the GeoRasterizer class can rasterize the GeoDataset with the defined cost
assumptions.

In [1]:
from pyorps import initialize_geo_dataset, save_empty_cost_assumptions, GeoRasterizer

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mNameError: name '_calculate_source_region_bounds' is not defined[0m
[0m[1mDuring: Pass nopython_type_inference[0m

#### 2. Download Vector Data from a WFS Server

A Web Feature Service (WFS) allows users to request and retrieve vector geospatial data (e.g., shapefiles) over the web. This is particularly useful for accessing up-to-date geospatial datasets.
How to Make a Proper WFS Request:
- Define the WFS server URL.
- Specify the layer or dataset you want to download.
- Use appropriate query parameters (e.g., bounding box, CRS, or feature type) to refine your request.

In this example we use an open-source dataset from the [Hessian State Office for Land Management and Geoinformation](https://gds.hessen.de/INTERSHOP/web/WFS/HLBG-Geodaten-Site/de_DE/-/EUR/ViewDownloadcenter-Start?path=Geodatendienste/WFS%20ALKIS-Grundriss)

##### 2.1 Define WFS-Request
First, we define the WFS request by specifying the URL and the layer name.

In [2]:
base_file_url = "https://www.gds.hessen.de/wfs2/aaa-suite/cgi-bin/alkis/vereinf/wfs"

wfs_request = {
    "url": base_file_url,
    "layer": "ave_Nutzung",
}

##### 2.2 Create bounding box
A significant amount of data is available for the defined request! To download only a portion of this data,let's define a bounding box. It is crucial that this bounding box is in the same Coordinate Reference System (CRS) as the requested layer, as this ensures accurate data retrieval and alignment ("EPSG:25832" in this case).

In [3]:
box_tuple = (471200, 5592700, 474200, 5595700)

##### 2.3 Initialize GeoDataset and download the data from the WFS
Now, we will initialize the GeoDataset instance and load the data from its source using the defined request and
bounding box.

In [4]:
geo_dataset = initialize_geo_dataset(wfs_request, bbox=box_tuple)
geo_dataset.load_data()

#### 3. Save an Empty Cost Assumptions CSV File

Cost assumptions assign terrain-specific costs to different features in the vector dataset (e.g., forests, water bodies, roads). These costs are critical for least-cost path analysis.
Steps to create the CSV file:
- Generate an empty CSV file with the required structure (e.g., columns for feature type and cost values).
- Save the file locally and  fill it with your specific cost assumptions.
##### 3.1 Define a file location to save the CSV file
Define a path string to the location where to save the CSV file. If the folder data/cost_assumptions does not exist,
create it beforehand.

In [5]:
save_path_empty_cost_assumptions = r"./data/cost_assumptions/cost_assumptions_template_HE_ALKIS.csv"

##### 3.2 Generate and save an empty cost assumptions file with zero values for the geospatial dataset.
This process involves analyzing the dataset to identify relevant feature columns and creating a CostAssumptions
object that assigns zero costs to all features. The resulting CSV file will serve as a template to put in your
specific cost values.

In [6]:
save_empty_cost_assumptions(geo_dataset, save_path_empty_cost_assumptions)

{('Halde', nan): 0,
 ('Straßenverkehr', nan): 0,
 ('Weg', nan): 0,
 ('Weg', 'Fußweg'): 0,
 ('Landwirtschaft', 'Ackerland'): 0,
 ('Landwirtschaft', 'Grünland'): 0,
 ('Landwirtschaft', 'Gartenbauland'): 0,
 ('Wald', 'Laub- und Nadelholz'): 0,
 ('Wald', 'Nadelholz'): 0,
 ('Wald', 'Laubholz'): 0,
 ('Unland/Vegetationslose Fläche', nan): 0,
 ('Fließgewässer', 'Graben'): 0,
 ('Gehölz', nan): 0,
 ('Industrie- und Gewerbefläche', 'Wasserwerk'): 0,
 ('Fließgewässer', nan): 0,
 ('Platz', nan): 0,
 ('Sport-, Freizeit- und Erholungsfläche', 'Sportanlage'): 0,
 ('Sport-, Freizeit- und Erholungsfläche',
  'Gebäude- und Freifläche Sport, Freizeit und Erholung'): 0,
 ('Fläche gemischter Nutzung', 'Landwirtschaftliche Betriebsfläche'): 0,
 ('Fläche gemischter Nutzung', 'Wohnen'): 0,
 ('Wohnbaufläche', nan): 0,
 ('Fläche gemischter Nutzung',
  'Gebäude- und Freifläche Land- und Forstwirtschaft'): 0,
 ('Fläche gemischter Nutzung',
  'Gebäude- und Freifläche Mischnutzung mit Wohnen'): 0,
 ('Fläche gemisch

##### 3.3 Use the template CSV file for your cost assumptions
Fill the cost column of the previously generated CSV file with your cost assumptions and save it as a new file!

#### 4. Rasterize the Vector Dataset Using Cost Assumptions

Rasterization converts vector data (e.g. polygons) into a raster format (gridded data). Each pixel in the raster is assigned a value based on the cost assumptions.
Steps to Rasterize the Data:
- Create a GeoRasterizer object with the vector dataset and the cost assumptions CSV file.
- Rasterize the vector data, assign pixel values based on the cost assumptions and save the raster file.

##### 4.1 Create the GeoRasterizer instance
Create an instance of the GeoRasterizer class with the GeoDataset instance and the path to the cost file, filled
with your cost assumptions.

In [7]:
cost_file_path = r"./data/cost_assumptions/cost_assumptions.csv"
rasterizer = GeoRasterizer(geo_dataset, cost_file_path)

##### 4.2 Rasterize the vector data and save the raster data
Rasterize the vector data by calling the rasterize method with the GeoRasterizer instance. Pass a path to the
save_path parameter to directly save the raster file after it has been created.

In [8]:
raster_save_path = r"./data/raster/big_raster.tiff"
rasterizer.rasterize(save_path=raster_save_path)

##### You can now use the rasterized geodata for your route planning tasks with PYORPS! Refer to the minimal_example.ipynb for guidance.